In [ ]:
import pandas as pd
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



In [ ]:
# Load the crime dataset and remove crimes that are not of immediate interest
crime_df = pd.read_csv("/Users/someshsahu/Documents/DSAI Project/crimes.csv")

crime_counts = crime_df['OFNS_DESC'].value_counts()
filtered_df = crime_df[crime_df['OFNS_DESC'].isin(crime_counts[crime_counts > 1000].index)]



removed_filtered_df = filtered_df.drop(filtered_df[filtered_df['OFNS_DESC'].isin(['VOIDED ARRESTS', 'OTHER STATE LAWS', 'FORGERY', 'FRAUDS', 'OFFENSES INVOLVING FRAUD', 'INTOXICATED & IMPAIRED DRIVING', 'OTHER TRAFFIC INFRACTION', 'INTOXICATED/IMPAIRED DRIVING', 'GAMBLING', 'THEFT-FRAUD', 'MOVING INFRACTIONS', 'ADMINISTRATIVE CODE', 'OTHER STATE LAWS (NON PENAL LA'])].index)
removed_filtered_df.columns
# new_df = removed_filtered_df.iloc[0:100]


The code below provides us with the top 3 precincts in terms of average number of crimes in a given month

In [ ]:


removed_filtered_df['ARREST_DATE'] = pd.to_datetime(removed_filtered_df['ARREST_DATE'], format='%m/%d/%Y')
removed_filtered_df['year_month'] = removed_filtered_df['ARREST_DATE'].dt.to_period('M')
arrests_by_precinct = removed_filtered_df.groupby(['ARREST_PRECINCT', 'year_month']).size().reset_index(name='arrest_count')

# loop through each period and identify the top 3 precincts with the highest crime rates
top_precincts = pd.DataFrame(columns=['year_month', 'top_precincts'])
for period in arrests_by_precinct['year_month'].unique():
    period_data = arrests_by_precinct[arrests_by_precinct['year_month'] == period]
    period_data['avg_arrests'] = period_data['arrest_count'].mean()
    precincts = period_data.sort_values(by=['arrest_count'], ascending=False)['ARREST_PRECINCT'][:3]
    top_precincts = top_precincts.append({'year_month': period, 'top_precincts': precincts.tolist()}, ignore_index=True)

# output the top precincts for each period
print(top_precincts)

In [ ]:
# group the data by period of time and calculate the total number of arrests in each group
removed_filtered_df['ARREST_DATE'] = pd.to_datetime(removed_filtered_df['ARREST_DATE'], format='%m/%d/%Y')
removed_filtered_df['year_month'] = removed_filtered_df['ARREST_DATE'].dt.to_period('M')
arrests_by_period = removed_filtered_df.groupby(['year_month']).size().reset_index(name='arrest_count')

# perform time series analysis
arrests_by_period.set_index('year_month', inplace=True)
arrests_by_period.index = pd.to_datetime(arrests_by_period.index.to_timestamp())
decomposition = sm.tsa.seasonal_decompose(arrests_by_period['arrest_count'], model='additive', period=12)

# plot the results of the time series analysis
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(12, 8))
decomposition.observed.plot(ax=ax1)
ax1.set_ylabel('Observed')
decomposition.trend.plot(ax=ax2)
ax2.set_ylabel('Trend')
decomposition.seasonal.plot(ax=ax3)
ax3.set_ylabel('Seasonal')
decomposition.resid.plot(ax=ax4)
ax4.set_ylabel('Residual')
plt.show()

In [ ]:

# Create a subset of the data with relevant features
arrests_subset = arrests_df[['ARREST_PRECINCT', 'year_month', 'num_arrests']]

# Create a pivot table to aggregate the data by precinct and period of time
arrests_pivot = pd.pivot_table(arrests_subset, values='num_arrests', index=['ARREST_PRECINCT'], columns=['year_month'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(arrests_pivot.dropna().drop(['top_3'], axis=1), arrests_pivot.dropna()['top_3'], test_size=0.2, random_state=42)

# Train a decision tree regression model
model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)

# Evaluate the performance of the model on the testing set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


In [ ]:
street_map = gpd.read_file('/Users/someshsahu/Downloads/Shapefiles and base map/geo_export_e32126a9-4611-4f2e-a62d-d39fdd22619e.shp')
street_map.to_crs(4326)


In [ ]:
geometry21 = [Point(xy) for xy in zip(new_df['Longitude'], new_df['Latitude'])]
geo_df21 = gpd.GeoDataFrame(new_df, crs="EPSG:4326", geometry=geometry21)
geo_df21

#plot both streetmap and and long/lat points
fig, ax = plt.subplots(figsize=(15,15))
geo_df21.plot(column="OFNS_DESC", ax=ax,alpha=0.5, legend=True,markersize=30)
street_map.to_crs(4326).plot(ax=ax, alpha=0.4,color='grey')
# add title to graph
plt.title('Crime in NYC', fontsize=15,fontweight='bold')
# set latitiude and longitude boundaries for map display
plt.xlim(-74.3, -73.7)
plt.ylim(40.5, 40.95)
plt.show()